# This will scrape GoodReads using their API and BS4 

In [2]:
from GRRatingScraper import *


ImportError: cannot import name 'scrapeSearchMultiPages'

In [27]:
key = 'OvwVfE5FFyCu0IpiwxWGVQ'

In [28]:
search_key = 'https://www.goodreads.com/search/index.xml?key=' + key + '&q='

In [29]:
movies_based_on_books = 'http://www.imdb.com/search/keyword?keywords= \
                         based-on-novel&mode=advanced&page=1&title_type= \
                         movie&ref_=kw_vw_adv&sort=user_rating,desc'

In [30]:
movies_based_on_books_05_15 = 'http://www.imdb.com/search/keyword?keywords=based-on-book&\
                                pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=2328987022&pf_rd_r=\
                                1B8PBQRZRA1N3R10W1ZK&pf_rd_s=center-1&pf_rd_t=15051&\
                                pf_rd_i=moka&ref_=kw_vw_smp&sort=user_rating,desc&mode=simple&\
                                page=1&title_type=movie&release_date=2005%2C2015'

In [31]:
def readPage(search_page_url):
    response = requests.get(search_page_url)
    page = response.text
    soup = BeautifulSoup(page, 'xml')
  
    return soup

## Test for getting goodreads data

In [32]:
page_data = readPage('https://www.goodreads.com/search/index.xml?key=OvwVfE5FFyCu0IpiwxWGVQ&q=Hababam%20Sinifi')

In [33]:
#print(page_data)

In [34]:
q = search_key + urllib.parse.quote('Hababam Sinifi')

## Get movie book list from IMDB

In [38]:
movie_book_list = scrapeSearchMultiPages(movies_based_on_books, 10)

In [39]:
movie_book_list = [movie for movie,_ in movie_book_list]

In [40]:
print(movie_book_list[:5])

['Hababam Sinifi', 'The Godfather', 'National Theatre Live: Frankenstein', 'The Godfather: Part II', 'Hababam Sinifi Sinifta Kaldi']


## Use the IMDB movie list to get data on the books the movies were based on

Takes the top hit and pulls the data from it, if there are no results, nothing is recorded.

In [41]:
def getBookData(movie_book_list, key=key, search_key=search_key):
    movie_book_names = []
    movie_book_ratings = []
    movie_book_ratings_counts = []
    
    got_books, missed_books = 0, 0

    for movie in movie_book_list:
        page_data = readPage(search_key + urllib.parse.quote(movie))
        total_results = page_data.find('total-results').text
        if int(total_results) > 1:
            rating = float(page_data.find('average_rating').text)
            rating_count = int(page_data.find('ratings_count').text)
            
            movie_book_names.append(movie)
            movie_book_ratings.append(rating)
            movie_book_ratings_counts.append(rating_count)
            got_books += 1
            print('Got {} books'.format(str(got_books)))
        else:
            missed_books += 1
            print('Missed {} books'.format(str(missed_books)))
        
        time.sleep(1)
            
    return [movie_book_names, movie_book_ratings, movie_book_ratings_counts]
        
        

In [42]:
book_data = getBookData(movie_book_list)

Got 1 books
Got 2 books
Missed 1 books
Got 3 books
Missed 2 books
Missed 3 books
Got 4 books
Got 5 books
Missed 4 books
Got 6 books
Missed 5 books
Missed 6 books
Got 7 books
Got 8 books
Missed 7 books
Got 9 books
Missed 8 books
Got 10 books
Got 11 books
Got 12 books
Got 13 books
Missed 9 books
Missed 10 books
Got 14 books
Got 15 books
Missed 11 books
Missed 12 books
Missed 13 books
Missed 14 books
Got 16 books
Missed 15 books
Missed 16 books
Missed 17 books
Got 17 books
Got 18 books
Got 19 books
Got 20 books
Got 21 books
Got 22 books
Got 23 books
Got 24 books
Missed 18 books
Got 25 books
Got 26 books
Missed 19 books
Got 27 books
Got 28 books
Missed 20 books
Got 29 books
Got 30 books
Got 31 books
Got 32 books
Missed 21 books
Got 33 books
Missed 22 books
Missed 23 books
Got 34 books
Got 35 books
Missed 24 books
Missed 25 books
Got 36 books
Got 37 books
Got 38 books
Got 39 books
Got 40 books
Got 41 books
Got 42 books
Got 43 books
Got 44 books
Got 45 books
Got 46 books
Got 47 books
Got 48 

In [43]:
print(book_data)

[['Hababam Sinifi', 'The Godfather', 'The Godfather: Part II', 'Fight Club', 'The Lord of the Rings: The Return of the King', 'Morometii', 'The Lord of the Rings: The Fellowship of the Ring', 'Forrest Gump', 'Selvi boylum, al yazmalim', 'Goodfellas', "One Flew Over the Cuckoo's Nest", 'The Lord of the Rings: The Two Towers', 'Harakiri', 'The Silence of the Lambs', "It's a Wonderful Life", 'La Celestina', 'The Prestige', 'The Green Mile', 'Psycho', 'Apocalypse Now', 'Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb', 'Grave of the Fireflies', 'Paths of Glory', 'Woman in the Dunes', 'Le trou', 'Guide', 'Cel mai iubit dintre pamînteni', 'Brown of Harvard', "Devil's Bride", 'Forest of the Hanged', 'The Golden Calf', 'When Knighthood Was in Flower', 'Janice Meredith', 'Hotaru no hikari', 'Anugraham', 'Requiem for a Dream', 'The Shining', 'A Clockwork Orange', 'Once Upon a Time in America', 'To Kill a Mockingbird', 'Vertigo', 'Das Boot', '3 Idiots', 'Indignation', 'Double

In [44]:
def generateGRDataFrame(movie_book_data):
    header = ['Title', 'GRRating', 'GRRatingCount']
    movie_book_data_dict = defaultdict(list)

    for _ in range(len(header)):
        movie_book_data_dict[header[_]] = movie_book_data[_]
        
    movie_book_df = pd.DataFrame(movie_book_data_dict)
    movie_book_df = movie_book_df[header]
    
    return movie_book_df

In [45]:
def writeGRToCSV(movie_book_data, outfilename):
    movie_book_df = generateGRDataFrame(movie_book_data)
    
    #print(movie_df.head())
    
    movie_book_df.to_csv(outfilename)

In [46]:
writeGRToCSV(book_data, 'movie_books_GR_ratings.csv')

## Pickle movie list

In [47]:
import pickle

In [48]:
new_book_data = book_data[0]

In [49]:
with open('pickles/GR_movie_book_list.pkl', 'wb') as picklefile:
    pickle.dump(new_book_data, picklefile)